In [1]:
# import socket

# # Server configuration
# SERVER_HOST = '127.0.0.1'
# SERVER_PORT = 1235

# sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# while True:
#     domain = input("Enter domain name: ").strip()
#     if not domain:
#         break

#     sock.sendto(domain.encode(), (SERVER_HOST, SERVER_PORT))
#     data, _ = sock.recvfrom(1024)
#     print(f"DNS Response: {data.decode()}")



import socket
from scapy.all import rdpcap, DNS
from datetime import datetime

SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5553

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Read PCAP and filter DNS queries
packets = rdpcap("7.pcap") ###

dns_packets = [pkt for pkt in packets if pkt.haslayer(DNS)]


In [16]:
for idx, pkt in enumerate(dns_packets):
    dns_payload = pkt[DNS]
    domain = "unknown"
    qtype = 0
    if dns_payload.qd:
        domain = dns_payload.qd.qname.decode().rstrip('.')
        qtype = dns_payload.qd.qtype

    # Create custom header HHMMSS + ID (2 digits)
    now = datetime.now()
    header = f"{now.hour:02}{now.minute:02}{now.second:02}{idx:02}".encode()  # 8 bytes
    dns_payload = bytes(pkt[DNS])
    # Prepend custom header
    message = header + dns_payload

    # Send to server
    sock.sendto(message, (SERVER_HOST, SERVER_PORT))

    data, _ = sock.recvfrom(1024)
    print(f"Response for packet {idx} : {qtype} : {domain} :  {data.decode()}")

Response for packet 0 : 12 : _apple-mobdev._tcp.local :  192.168.1.9
Response for packet 1 : 12 : _apple-mobdev._tcp.local :  192.168.1.10
Response for packet 2 : 0 : unknown :  192.168.1.6
Response for packet 3 : 0 : unknown :  192.168.1.7
Response for packet 4 : 0 : unknown :  192.168.1.8
Response for packet 5 : 0 : unknown :  192.168.1.9
Response for packet 6 : 0 : unknown :  192.168.1.10
Response for packet 7 : 0 : unknown :  192.168.1.6
Response for packet 8 : 0 : unknown :  192.168.1.7
Response for packet 9 : 0 : unknown :  192.168.1.8
Response for packet 10 : 1 : wikipedia.org :  192.168.1.9
Response for packet 11 : 33 : Brother MFC-7860DW._pdl-datastream._tcp.local :  192.168.1.10
Response for packet 12 : 33 : Brother MFC-7860DW._pdl-datastream._tcp.local :  192.168.1.6
Response for packet 13 : 1 : reddit.com :  192.168.1.7
Response for packet 14 : 33 : Brother MFC-7860DW._pdl-datastream._tcp.local :  192.168.1.8
Response for packet 15 : 33 : Brother MFC-7860DW._pdl-datastream.

In [24]:

import socket
import dpkt
from datetime import datetime

SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5553

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

dns_packets = []
# Open PCAP file
with open("7.pcap", "rb") as f:
    pcap = dpkt.pcap.Reader(f)

    for ts, buf in pcap:    
        try:
            eth = dpkt.ethernet.Ethernet(buf)
            if not isinstance(eth.data, dpkt.ip.IP):
                continue
            ip = eth.data
            if not isinstance(ip.data, dpkt.udp.UDP):
                continue
            udp = ip.data

            # Check if it's DNS (port 53) and parse
            if udp.sport == 53 or udp.dport == 53:
                dns = dpkt.dns.DNS(udp.data)
                if dns.qr == dpkt.dns.DNS_Q:  # Query
                    dns_packets.append(dns)
        except Exception:
            continue

# Send DNS packets to server
for idx, dns in enumerate(dns_packets):
    # Raw DNS payload
    # print(type(dns))
    dns_payload = bytes(dns)

    # Custom header HHMMSS + ID
    now = datetime.now()
    header = f"{now.hour:02}{now.minute:02}{now.second:02}{idx:02}".encode()  # 8 bytes

    # Prepend custom header
    message = header + dns_payload
    sock.sendto(message, (SERVER_HOST, SERVER_PORT))

    # Receive response
    data, _ = sock.recvfrom(4096)

    # Print query name(s) for debugging
    query_names = [q.name for q in dns.qd] if dns.qd else []
    print(f"Response for packet {idx}: Query={query_names} -> {data.decode()}")


Response for packet 0: Query=['wikipedia.org'] -> 192.168.1.6
Response for packet 1: Query=['reddit.com'] -> 192.168.1.7
Response for packet 2: Query=['apple.com'] -> 192.168.1.8
Response for packet 3: Query=['twitter.com'] -> 192.168.1.9
Response for packet 4: Query=['yahoo.com'] -> 192.168.1.10
Response for packet 5: Query=['linkedin.com'] -> 192.168.1.6
